Dataset-dataset berikut ini menunjukkan persentase lemak, protein, dan food yang dikonsumsi dari setiap jenis makanan yang terdaftar. Akhir dataset juga mencakup persentase obesitas dan kurang gizi, dan persentase kasus COVID-19 Terkonfirmasi/Kematian/Sembuh/Aktif (Catatan: Semua data memiliki unit% kecuali Populasi, Unit % tersebut berdasarkan jumlah populasi).
Dataset yang saya gunakan dari https://www.kaggle.com/mariaren/covid19-healthy-diet-dataset/notebooks

**Import Library dan Dataset yang dibutuhkan menggunakan pandas dan menyimpannya kedalam variabel yang mudah diingat**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import Semua Data
df_food = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_Quantity_kg_Data.csv')
df_protein = pd.read_csv('../input/covid19-healthy-diet-dataset/Protein_Supply_Quantity_Data.csv')
df_fat = pd.read_csv('../input/covid19-healthy-diet-dataset/Fat_Supply_Quantity_Data.csv')
df_category = pd.read_csv('../input/covid19-healthy-diet-dataset/Supply_Food_Data_Descriptions.csv')

**Memeriksa 5 data teratas dari tiap dataset**

In [ ]:
df_food.head()

In [ ]:
df_protein.head()

In [ ]:
df_fat.head()

In [ ]:
df_category.head()

**Memisahkan antara column tentang makanan yang di supply dan bukan makanan**

di akhir kita mempunyai data list berisi semua column tentang supply makanan dan non supply makanan atau column tentang covid-19 beserta populasi, cara kalian mendapatkannya bebas yang penting sesuai output.
Hasil Akhir Yang di harapkan :

1. supply columns => columns berisi column tentang semua jenis supply makanan
2. all columns => Semua columns
3. non supply columns => column column tentang data covid-19 + obesitas + Undernourished + populasi



In [ ]:
# menempatkan semua data kedalam all_columns
all_columns = df_food.columns.tolist()
print(f'All Columns : {all_columns} \n\n')

# menempatkan data yang merupakan kategori food dalam supply_columns
supply_columns = df_category['Categories'].tolist()
print(f'Supply Columns : {supply_columns} \n\n')

# menempatkan data yang merupakan kategori non food dalam non_supply_columns
non_supply_columns = [i for i in all_columns if i not in supply_columns]
print(f'Non-Supply Columns : {non_supply_columns}')

**Menggabungkan DataFrame**

Dengan menggabungkan data yang merupakan kategori non food/non supply dengan kategori supply dan data yang merupakan supply dari tiap df dibedakan dengan nama daf nya misalkan 'Meat_food', 'Meat_fat, dan 'Meat_protein'

In [ ]:
# menyimpan data yang merupakan non supply
df_non_supply_new = df_food[non_supply_columns]
# menyimpan data food yang merupakan supply dan ditambahkan kolom country 
df_food_new = df_food[['Country']+supply_columns]
# menyimpan data protein yang merupakan supply dan ditambahkan kolom country
df_protein_new = df_protein[['Country']+supply_columns]
# menyimpan data fat yang merupakan supply dan ditambahkan kolom country
df_fat_new = df_fat[['Country']+supply_columns]

In [ ]:
# Menggabungkan dataset yang sudah disimpan dengan variabel di atas dan menambahkan suffix berbeda untuk setiap supply dataset menggunakan pd.merge
df1=pd.merge(df_food_new, df_protein_new, on="Country", suffixes=('_food', '_protein'), how = 'outer')
# Menambahkan _fat 
df_fat_new_cols = [i + '_fat' if i != 'Country' else i for i in df_fat_new.columns]
df_fat_new.columns = df_fat_new_cols

df2=pd.merge(df_fat_new, df_non_supply_new, how='outer', on='Country')
df3=pd.merge(df1, df2, how='left', on='Country')
df3.columns.tolist()

In [ ]:
df3

**Sorting and Filtering**

Buatlah sebuah data atau gunakan data gabungan dari data diatas, dimana data ini di sort dari yang terbesar ke yang terkecil berdasarkan column Obesitas.

Kemudian Lakukan Perintah Berikut :

1. print data hasil sorting
2. Berapa persen jumlah obesitas di Indonesia berdasarkan populasinya? kemudian tunjukan cara mendapatkannya!
3. Urutan ke berapakah Indonesia mengenai proposi penduduk yang obesitas berdasarkan total populasinya dari yang terbesar? kemudian tunjukan cara mendapatkannya!

In [ ]:
# Urutkan (sorting) dan print hasil sorting
df_sort = df3.sort_values(by=['Obesity'], ascending=False, ignore_index=True)
df_sort

In [ ]:
# code untuk mendapatkan nilai proporsi obesitas di Indonesia
obesitas_persen = list(df_sort['Obesity'][df_sort['Country']=='Indonesia'])[0]
obesitas = list(df_sort['Obesity'][df_sort['Country']=='Indonesia']*df_sort['Population'][df_sort['Country']=='Indonesia']*0.01)[0]

print(f'Jumlah yang mengidap obesitas di Indonesia ada {round(obesitas_persen, 10)}% dari jumlah populasi.') # 10 digit
print(f'Jumlah yang mengidap obesitas di Indonesia ada {round(obesitas)} orang.')

In [ ]:
# code untuk mendapatkan urutan indoensia mengenai proposi obesitas dari terbesar
indo = list(df_sort[df_sort['Country']== 'Indonesia'].index+1)[0]

print('Indonesia urutan : {}'.format(indo))

**Binning Data, Grouping, dan Aggregating**

Binning data Terkonfirmasi COVID-19 (column Confirmed) dari data food supply menjadi 5 bagian berdasarkan quartile. kalian dapat menggunakan fungsi qcut untuk melakukan bininng ini. Masukan data hasil binning tersebut menjadi column baru untuk data food supply. misalkan beri nama column tersebut dengan 'Confirmed Category'. Berikan label 'Sangat Tidak Banyak', 'Tidak Banyak', 'Sedang', 'Cukup Banyak', 'Sangat Banyak'. untuk masing-masing binning dari interval yang memiliki nilai terkecil sampai yang terbesar

Baca dokumentasi berikut untuk fungsi qcut : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html

1. Buat data baru yang berupa data grouping semua data berdasarkan column Hasil Bininng atau column 'Confirmed Category'.
2. Grouping hanya column 'Obesity', 'Meat', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products', 'Vegetal Products' saja.
3. Kemudian Aggregate dengan fungsi mean.

In [ ]:
# Melakukan binning pada kolom terkonfirmasi untuk DataFrame df_fat
labels = ['Sangat Tidak Banyak', 'Tidak Banyak', 'Sedang', 'Cukup Banyak', 'Sangat Banyak']

df_fat['Confirmed Category'] = pd.qcut((df_fat['Confirmed']), 5, labels=labels)

In [ ]:
# Melakukan grouping pada kolom hasil binning ('Confirmed Category'), kemudian aggregate menggunakan fungsi mean
cols = ['Obesity', 'Meat', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products', 'Vegetal Products']

groupby_confirmed = df_fat.groupby('Confirmed Category')[cols].mean()
groupby_confirmed

In [ ]:
# Menyimpan DataFrame hasil binning, grouping, dan aggregating dalam bentuk csv
groupby_confirmed.to_csv('groupby_confirmed.csv')

**Matplotlib Visualization: Bar plot**

Gunakan data grouping pada soal sebelumnya untuk melakukan visualisasi.

Normalisasi data hasil grouping tersebut dengan rumus min-max normalisasi. Kemudian buatlah grouping Visualization dengan berplot, visualisasikan data column 'Obesity', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products' dan 'Vegetal Products'.

In [ ]:
# Gunakan DataFrame hasil grouping 
df = groupby_confirmed

# Lihat df 
display(df)

In [ ]:
# Normalisasi pada DataFrame df
df_norm = (df - df.min()) / (df.max() - df.min())

# Lihat df_norm dan pastikan data telah ternormalisasi
display(df_norm)

In [ ]:
# Buat bar plot dari df_norm 
x = np.arange(len(df_norm.columns))
y = df_norm.loc['Sangat Tidak Banyak']
y1 = df_norm.loc['Tidak Banyak']
y2 = df_norm.loc['Sedang']
y3 = df_norm.loc['Cukup Banyak']
y4 = df_norm.loc['Sangat Banyak']

In [ ]:
fig, ax = plt.subplots(figsize=(14,10))
width = 0.1
a1= ax.bar(x, y, width, label='Sangat Tidak Banyak')
a2= ax.bar(x+0.1, y1, width, label='Tidak Banyak')
a3= ax.bar(x+0.2, y2, width, label='Sedang')
a4= ax.bar(x+0.3, y3, width, label='Cukup Banyak')
a5= ax.bar(x+0.4, y4, width, label='Sangat Banyak')

#menambahkan label
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        plt.text(rect.get_x() + rect.get_width() / 2, height, str(round(height,2)),
                    ha='center', va='bottom')
        
autolabel(a1)
autolabel(a2)
autolabel(a3)
autolabel(a4)
autolabel(a5)

plt.xticks(x + 0.2, ('Obesity', 'Meat', 'Fruits - Excluding Wine', 'Fish, Seafood', 'Animal Products','Vegetal Products'))
ax.set_title('Perbandingan Konsumsi Makanan Berdasarkan Tingkat Kasus COVID 19')
ax.legend(bbox_to_anchor=(0.95,1.1))
plt.show()

**Matplotlib Visualization: Scatter Plot**

Buatlah visualisasi perbandingan scatter plot dimana kita akan membandingkan respon Jumlah Kasus Covid-19 yang terkonfirmasi berdasarkan Supply Sayuran. dimana:

Tandai proporsi kematian jumlah kematian (column Death) dengan ukuran dari data poin.
Tandai proporsi produk hewani (column Animal Product) dengan warna, berikan colormap 'gist_yarg'.

In [ ]:

fig, ax = plt.subplots(figsize=(18, 12))

V = df_food['Vegetal Products']
C = df_food['Confirmed']
A = df_food['Animal Products']
# mengalikan kolom death agar dapat terlihat dalam visualisasi
D = 200000*df_food['Deaths']

# menggunakan kolom persentase kematian ('Death') sebagai ukuran pointnya, produk hewani ('Animal Products') sebagai warna point dengan cmap 'plasma'.
map = plt.scatter(V,C, c=A, cmap='plasma', sizes=D)
sns.regplot(x="Vegetal Products", y="Confirmed", data=df_food, scatter=False, color='black');

# axis labeling
ax.set_xlabel('Vegetal Products')
ax.set_ylabel('Confirmed')
ax.set_title('Healthy Food and Covid-19')

fig.colorbar(map)
plt.show()